In [2]:
import os
import pydicom
import numpy as np
from PIL import Image
import openslide as openslide
import scripts.conversion
import scripts.train
import sys
from PIL.GimpGradientFile import linear
%load_ext autoreload
%autoreload 2
from tqdm import tqdm
from IPython.display import clear_output
sys.path.append('.')

In [ ]:
scripts.conversion.Conversiontopng.convert_dicom_svs_to_png(
    input_folder="data\\H&E-stained",
    output_folder="output_dsfalse",
    svs_level=0,
    auto_downsample=False,
    max_images=1,
)


🚀 CUDA acceleration enabled on NVIDIA GeForce RTX 4060 Laptop GPU
💾 CUDA memory available: 8.0 GB
🔧 Using 15 worker threads
📁 Found 1 files to process (recursive)


🔄 Converting:   0%|          | 0/1 [00:00<?, ?file/s]

In [15]:
trainer = scripts.train.quick_train(
    data_dir='data/',
    model_name='resnet50',
    num_classes=3,
    num_epochs=1,
    patch_size=1000,
    patches_per_slide=500,
    batch_size=8
)


🚀 Starting quick training with resnet50


C:\Users\tanth\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\tanth\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\tanth\Desktop\Pathothai\scripts\train.py:167: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler() if mixed_precision else None


Found 209 slides across 2 classes
Resumed from epoch 0
Starting training on cuda
Model: resnet50
Training samples: 83500
Validation samples: 21000


Epoch 0:   0%|          | 0/10438 [00:00<?, ?it/s]c:\Users\tanth\Desktop\Pathothai\scripts\train.py:310: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 0:   0%|          | 2/10438 [00:31<45:23:24, 15.66s/it, Loss=0.2893, Acc=93.75%]


Training interrupted by user

Training interrupted by user


UnboundLocalError: cannot access local variable 'val_acc' where it is not associated with a value